In [100]:
# @title
import warnings
warnings.filterwarnings('ignore')


try:
    import cirq
except ImportError:
    print('installing cirq...')
    %pip install cirq --quiet
    import cirq
    print('installed cirq.')


#!git clone https://github.com/the-codingschool/bb84.git
#from bb84.bb84 import BB84

import matplotlib.pyplot as plt
import random

In [101]:
#Process 1: Sending

#Step 1: Alice Generates a Random Key


def generate_classical_key(num_bits): 
    #Step 1: Alice generates a random secret key of classical bits
    key = [random.choice(['0','1']) for _ in range(num_bits)]
    return ''.join (key)

def alice_prepare_bb84(key, num_bits):
    #Create one qubit per bit
    qubits = [cirq.NamedQubit(f'a{i}') for i in range(num_bits)]
    circuit = cirq.Circuit()
    bases = []

    #Convert classical bits to qubit states and apply random bases
    for i, (qubit, bit) in enumerate(zip(qubits,key)):
        basis = random.choice([0,1]) #0: rectilinear(Z), 1:diagonal(X)
        bases.append(basis)

        #Convert classical bit to qubit state
        if bit == '1':
            circuit.append(cirq.X(qubit))

        #Apply H-gate for diagonal basis
        if basis == 1:
            circuit.append(cirq.H(qubit)) 
            #Applies H-gate for bit randomly chosen to be in X-basis
    return circuit, bases, qubits

#Generate classical key
num_bits= 16
secret_key = generate_classical_key(num_bits)
print(f"Alice's Classical Secret Key ({num_bits} bits): {secret_key}")

#Alice prepares quantum states
alice_circuit, alice_bases, qubits = alice_prepare_bb84(secret_key, num_bits)
print("\nAlice's Basis Choices (0=Rectilinear, 1=Diagonal):", alice_bases)

#Print the Circuit
print("\nAlice's Circuit for All Bits:")
print(alice_circuit)

#Print state preparation for each bit
print("\nState Preparation for Each Bit:")
for i, (bit, basis) in enumerate(zip(secret_key, alice_bases)):
    print(f"Bit {i} (key={bit}, basis={basis}): q{i}")

Alice's Classical Secret Key (16 bits): 1100101001001110

Alice's Basis Choices (0=Rectilinear, 1=Diagonal): [1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1]

Alice's Circuit for All Bits:
a0: ────X───H───

a1: ────X───────

a2: ────H───────

a3: ────H───────

a4: ────X───H───

a6: ────X───────

a7: ────H───────

a9: ────X───────

a10: ───H───────

a11: ───H───────

a12: ───X───────

a13: ───X───────

a14: ───X───────

a15: ───H───────

State Preparation for Each Bit:
Bit 0 (key=1, basis=1): q0
Bit 1 (key=1, basis=0): q1
Bit 2 (key=0, basis=1): q2
Bit 3 (key=0, basis=1): q3
Bit 4 (key=1, basis=1): q4
Bit 5 (key=0, basis=0): q5
Bit 6 (key=1, basis=0): q6
Bit 7 (key=0, basis=1): q7
Bit 8 (key=0, basis=0): q8
Bit 9 (key=1, basis=0): q9
Bit 10 (key=0, basis=1): q10
Bit 11 (key=0, basis=1): q11
Bit 12 (key=1, basis=0): q12
Bit 13 (key=1, basis=0): q13
Bit 14 (key=1, basis=0): q14
Bit 15 (key=0, basis=1): q15


In [102]:
#Process 2: Intercepting – Hannah

def eve_intercept_bb84(alice_circuit, qubits, num_bits):
    # Create Eve's probe qubits (one per Alice qubit)
    eve_probes = [cirq.NamedQubit(f'e{i}') for i in range(num_bits)]

    # Add entanglement operations to Alice's circuit
    for i in range(num_bits):
        qubit_0 = qubits[i]
        eve_probe = eve_probes[i]

        # Step 1: Eve prepares probe in |+⟩ = H|0⟩
        alice_circuit.append(cirq.H(eve_probe))

        # Step 2: Entangle with Alice's qubit using CNOT
        alice_circuit.append(cirq.CNOT(qubit_0, eve_probe))

        # Step 3: Eve stores the probe; original qubit continues to Bob

    return alice_circuit, eve_probes

# Eve intercepts and modifies Alice's circuit
alice_circuit, eve_probes = eve_intercept_bb84(alice_circuit, qubits, num_bits)

# Print modified circuit
print("\nCircuit After Eve's Interception (Process 2):")
print(alice_circuit)


Circuit After Eve's Interception (Process 2):
            ┌──────────────┐   ┌──┐
a0: ────X────H──────────────────@─────
                                │
a1: ────X─────@─────────────────┼─────
              │                 │
a2: ────H─────┼@────────────────┼─────
              ││                │
a3: ────H─────┼┼@───────────────┼─────
              │││               │
a4: ────X────H┼┼┼───────────────┼@────
              │││               ││
a5: ─────────@┼┼┼───────────────┼┼────
             ││││               ││
a6: ────X────┼┼┼┼@──────────────┼┼────
             │││││              ││
a7: ────H────┼┼┼┼┼@─────────────┼┼────
             ││││││             ││
a8: ─────────┼┼┼┼┼┼@────────────┼┼────
             │││││││            ││
a9: ────X────┼┼┼┼┼┼┼@───────────┼┼────
             ││││││││           ││
a10: ───H────┼┼┼┼┼┼┼┼@──────────┼┼────
             │││││││││          ││
a11: ───H────┼┼┼┼┼┼┼┼┼@─────────┼┼────
             ││││││││││         ││
a12: ───X────┼┼┼┼┼┼┼┼┼┼@────────┼

In [103]:
#Process 3: Receiving

def bob_measure_bb84(alice_circuit, qubits, num_bits):
    bob_bases = []
    bob_measurements = []

    #Creates random measurement in the Z or X basis
    for i in range(num_bits):
        qubit = qubits[i]
        bob_basis = random.choice([0, 1])  # 0: Z basis, 1: X basis
        bob_bases.append(bob_basis)

        # Create a new circuit for this qubit
        new_circuit = cirq.Circuit()

        # Includes Alice’s gates for this qubit
        for moment in alice_circuit:
            for op in moment:
                if qubit in op.qubits:
                    new_circuit.append(op)

        if bob_basis == 1:
            new_circuit.append(cirq.H(qubit))

        new_circuit.append(cirq.measure(qubit, key='m'))

        sim = cirq.Simulator()
        result = sim.run(new_circuit, repetitions=1)
        bit = result.measurements['m'][0][0]
        bob_measurements.append(bit)

    return bob_bases, bob_measurements


# Call Bob's measurement function
bob_bases, bob_measurements = bob_measure_bb84(alice_circuit, qubits, num_bits)

# Print Bob's measurement results
print("\nBob's Measurement Results:")
for i in range(num_bits):
    print(f"Bit {i} (basis={bob_bases[i]}, measurement={bob_measurements[i]}): a{i}")



Bob's Measurement Results:
Bit 0 (basis=0, measurement=1): a0
Bit 1 (basis=1, measurement=0): a1
Bit 2 (basis=0, measurement=1): a2
Bit 3 (basis=1, measurement=0): a3
Bit 4 (basis=0, measurement=0): a4
Bit 5 (basis=0, measurement=0): a5
Bit 6 (basis=1, measurement=1): a6
Bit 7 (basis=1, measurement=1): a7
Bit 8 (basis=0, measurement=0): a8
Bit 9 (basis=1, measurement=1): a9
Bit 10 (basis=0, measurement=1): a10
Bit 11 (basis=1, measurement=0): a11
Bit 12 (basis=1, measurement=1): a12
Bit 13 (basis=1, measurement=1): a13
Bit 14 (basis=1, measurement=1): a14
Bit 15 (basis=0, measurement=0): a15


In [104]:
# Process 4: Comparing (Reconciliation Stage)

def reconcile_keys(alice_bases, bob_bases, secret_key, bob_measurements, num_bits):
    sifted_key_alice = []
    sifted_key_bob = []
    matching_indices = []
    
    # Compare bases and bits for each position
    print("\nBit-by-Bit Comparison (Alice vs. Bob):")
    print("Index | Alice Bit | Alice Basis | Bob Bit | Bob Basis | Bases Match | Bits Match")
    print("-" * 70)
    for i in range(num_bits):
        alice_bit = secret_key[i]
        alice_basis = alice_bases[i]
        bob_bit = str(bob_measurements[i])
        bob_basis = bob_bases[i]
        bases_match = alice_basis == bob_basis
        bits_match = str(alice_bit == bob_bit) if bases_match else "-"
        
        # Print comparison with explicit string formatting
        print(f"{i:5d} | {alice_bit:9s} | {alice_basis:11d} | {bob_bit:7s} | {bob_basis:9d} | {str(bases_match):11s} | {str(bits_match):8s}")
        
        # Form sifted key for matching bases
        if bases_match:
            sifted_key_alice.append(alice_bit)
            sifted_key_bob.append(bob_bit)
            matching_indices.append(i)
    
    # Error checking: Calculate error rate
    errors = sum(1 for a, b in zip(sifted_key_alice, sifted_key_bob) if a != b)
    error_rate = errors / len(sifted_key_alice) if sifted_key_alice else 0
    return sifted_key_alice, sifted_key_bob, matching_indices, error_rate

sifted_key_alice, sifted_key_bob, matching_indices, error_rate = reconcile_keys(
    alice_bases, bob_bases, secret_key, bob_measurements, num_bits
)
print(f"\nSifted Key (Alice): {''.join(sifted_key_alice)}")
print(f"Sifted Key (Bob): {''.join(sifted_key_bob)}")
print(f"Matching Indices: {matching_indices}")
print(f"Error Rate: {error_rate:.3f}")


Bit-by-Bit Comparison (Alice vs. Bob):
Index | Alice Bit | Alice Basis | Bob Bit | Bob Basis | Bases Match | Bits Match
----------------------------------------------------------------------
    0 | 1         |           1 | 1       |         0 | False       | -       
    1 | 1         |           0 | 0       |         1 | False       | -       
    2 | 0         |           1 | 1       |         0 | False       | -       
    3 | 0         |           1 | 0       |         1 | True        | True    
    4 | 1         |           1 | 0       |         0 | False       | -       
    5 | 0         |           0 | 0       |         0 | True        | True    
    6 | 1         |           0 | 1       |         1 | False       | -       
    7 | 0         |           1 | 1       |         1 | True        | False   
    8 | 0         |           0 | 0       |         0 | True        | True    
    9 | 1         |           0 | 1       |         1 | False       | -       
   10 | 0         

In [105]:
# Process 5: Eve’s Measurements
def eve_measure_probes(alice_circuit, eve_probes, matching_indices, alice_bases, num_bits):
    eve_measurements = []
    for i in matching_indices:
        eve_probe = eve_probes[i]
        basis = alice_bases[i]  # Use the basis announced by Alice/Bob
        # Create a new circuit for Eve’s measurement
        eve_circuit = cirq.Circuit()
        # Include only Eve’s probe preparation and entanglement
        for moment in alice_circuit:
            for op in moment:
                if eve_probe in op.qubits:
                    eve_circuit.append(op)
        # Measure in the same basis as Alice/Bob
        if basis == 1:  # Diagonal (X) basis
            eve_circuit.append(cirq.H(eve_probe))
        eve_circuit.append(cirq.measure(eve_probe, key=f'e{i}'))
        sim = cirq.Simulator()
        result = sim.run(eve_circuit, repetitions=1)
        bit = result.measurements[f'e{i}'][0][0]
        eve_measurements.append(bit)
        eve_measurements = [int(x) for x in eve_measurements]
    return eve_measurements

eve_measurements = eve_measure_probes(alice_circuit, eve_probes, matching_indices, alice_bases, num_bits)
print(f"Eve’s Measurements for Matching Indices: {eve_measurements}")
# Compare Eve’s measurements with Alice’s sifted key
correct_guesses = sum(1 for e, a in zip(eve_measurements, sifted_key_alice) if str(e) == a)
info_gain = correct_guesses / len(sifted_key_alice) if sifted_key_alice else 0
print(f"Eve’s Information Gain (fraction of correct bits): {info_gain:.3f}")

Eve’s Measurements for Matching Indices: [0, 0, 0, 1, 0]
Eve’s Information Gain (fraction of correct bits): 0.800


In [106]:
#Process 6: Privacy Amplification
def privacy_amplification(sifted_key_alice, sifted_key_bob, matching_indices, final_key_length):
    # Select a random subset of sifted key bits for the final key
    if len(sifted_key_alice) < final_key_length:
        final_key_length = len(sifted_key_alice)
    
    indices = random.sample(range(len(sifted_key_alice)), final_key_length)
    amplified_key_alice = [sifted_key_alice[i] for i in indices]
    amplified_key_bob = [sifted_key_bob[i] for i in indices]
    amplified_indices = [matching_indices[i] for i in indices]
    
    return amplified_key_alice, amplified_key_bob, amplified_indices

# Call privacy amplification
final_key_length = max(1, len(sifted_key_alice) // 2)  # Half the sifted key length
amplified_key_alice, amplified_key_bob, amplified_indices = privacy_amplification(
    sifted_key_alice, sifted_key_bob, matching_indices, final_key_length
)
print("\nAmplified Key (Alice):", ''.join(amplified_key_alice))
print("Amplified Key (Bob):", ''.join(amplified_key_bob))
print("Amplified Indices:", amplified_indices)

# Fix output bug in state preparation (for clarity, not modifying original code)
print("\nState Preparation for Each Bit (Corrected Qubit Names):")
for i, (bit, basis, qubit) in enumerate(zip(secret_key, alice_bases, qubits)):
    print(f"Bit {i} (key={bit}, basis={basis}, qubit={qubit})")


Amplified Key (Alice): 00
Amplified Key (Bob): 00
Amplified Indices: [3, 5]

State Preparation for Each Bit (Corrected Qubit Names):
Bit 0 (key=1, basis=1, qubit=a0)
Bit 1 (key=1, basis=0, qubit=a1)
Bit 2 (key=0, basis=1, qubit=a2)
Bit 3 (key=0, basis=1, qubit=a3)
Bit 4 (key=1, basis=1, qubit=a4)
Bit 5 (key=0, basis=0, qubit=a5)
Bit 6 (key=1, basis=0, qubit=a6)
Bit 7 (key=0, basis=1, qubit=a7)
Bit 8 (key=0, basis=0, qubit=a8)
Bit 9 (key=1, basis=0, qubit=a9)
Bit 10 (key=0, basis=1, qubit=a10)
Bit 11 (key=0, basis=1, qubit=a11)
Bit 12 (key=1, basis=0, qubit=a12)
Bit 13 (key=1, basis=0, qubit=a13)
Bit 14 (key=1, basis=0, qubit=a14)
Bit 15 (key=0, basis=1, qubit=a15)
